# Zero-Noise Extrapolation (ZNE) — Pure Python Demo
This notebook runs a compact, dependency-free simulation of zero-noise extrapolation using only the Python standard library. It avoids external packages so the results can be reproduced in restricted environments while still providing a clear visual summary of the extrapolation process.


In [1]:
from pathlib import Path

from extrapolation_methods import linear_extrapolation_diagnostics
from unitary_folding import BASE_SEQUENCE, expectation_p0, run_folded_circuit


def generate_svg(xs, ys, slope, intercept, path):
    width, height = 520, 320
    padding = 60
    max_x = max(xs)
    min_y = min(min(ys), intercept)
    max_y = max(max(ys), intercept)
    yrange = max_y - min_y or 1

    def map_x(x):
        return padding + (width - 2 * padding) * (x / max_x)

    def map_y(y):
        return height - padding - (height - 2 * padding) * ((y - min_y) / yrange)

    lines = [
        f"<svg xmlns='http://www.w3.org/2000/svg' width='{width}' height='{height}' role='img'>",
        "<style>text{font-family:Arial, sans-serif;font-size:12px;} .tick{stroke:#555;stroke-width:1;} .curve{stroke:#1976d2;stroke-width:2;fill:none;} .scatter{fill:#d81b60;}</style>",
        f"<rect x='0' y='0' width='{width}' height='{height}' fill='white' stroke='none' />",
        f"<line x1='{padding}' y1='{height - padding}' x2='{width - padding}' y2='{height - padding}' class='tick' />",
        f"<line x1='{padding}' y1='{padding}' x2='{padding}' y2='{height - padding}' class='tick' />",
        f"<text x='{width / 2}' y='{height - 20}' text-anchor='middle'>Noise scaling factor (folds)</text>",
        f"<text x='20' y='{height / 2}' transform='rotate(-90 20 {height / 2})' text-anchor='middle'>P(|0⟩)</text>",
    ]

    for x in xs:
        X = map_x(x)
        lines.append(f"<line x1='{X}' y1='{height - padding}' x2='{X}' y2='{height - padding + 5}' class='tick' />")
        lines.append(f"<text x='{X}' y='{height - padding + 20}' text-anchor='middle'>{x}</text>")

    for frac in [0, 0.25, 0.5, 0.75, 1]:
        yval = min_y + yrange * frac
        Y = map_y(yval)
        lines.append(f"<line x1='{padding - 5}' y1='{Y}' x2='{padding}' y2='{Y}' class='tick' />")
        lines.append(f"<text x='{padding - 10}' y='{Y + 4}' text-anchor='end'>{yval:.2f}</text>")

    x0, x1 = 0, max_x
    y0, y1 = slope * x0 + intercept, slope * x1 + intercept
    lines.append(f"<line x1='{map_x(x0)}' y1='{map_y(y0)}' x2='{map_x(x1)}' y2='{map_y(y1)}' class='curve' stroke='#1976d2' />")

    for x, y in zip(xs, ys):
        lines.append(f"<circle cx='{map_x(x)}' cy='{map_y(y)}' r='4' class='scatter' />")

    zero_y = map_y(intercept)
    lines.append(f"<line x1='{map_x(0)}' y1='{zero_y}' x2='{map_x(max_x)}' y2='{zero_y}' stroke='#2e7d32' stroke-dasharray='6 4' />")
    lines.append(f"<text x='{map_x(0) + 5}' y='{zero_y - 8}' fill='#2e7d32'>Zero-noise estimate: {intercept:.4f}</text>")

    lines.append('</svg>')
    path.write_text('\n'.join(lines))


In [2]:
base_error = 0.04
scale_factors = [1, 3, 5]
results = []
for factor in scale_factors:
    rho = run_folded_circuit(base_error, factor, base_sequence=BASE_SEQUENCE)
    results.append(expectation_p0(rho))

fit = linear_extrapolation_diagnostics(scale_factors, results)
svg_path = Path('zne_results.svg')
generate_svg(scale_factors, results, fit['slope'], fit['intercept'], svg_path)

print('Zero-Noise Extrapolation demo (standard library simulation)')
print(f'Base depolarizing probability per gate: {base_error}')
print('Fold factor | P(|0>)')
print('-' * 28)
for factor, prob in zip(scale_factors, results):
    print(f'{factor:>5}      | {prob:.6f}')

print(f"Linear fit intercept (zero-noise): {fit['intercept']:.6f}")
print(f"Linear fit slope: {fit['slope']:.6f}, R^2: {fit['r2']:.6f}")
print(f'Plot saved to {svg_path.name}')


Zero-Noise Extrapolation demo (standard library simulation)
Base depolarizing probability per gate: 0.04
Fold factor | P(|0>)
----------------------------
    1      | 0.912289
    3      | 0.627507
    5      | 0.517559
Linear fit intercept (zero-noise): 0.981832
Linear fit slope: -0.098683, R^2: 0.938620
Plot saved to zne_results.svg


base_error = 0.04
scale_factors = [1, 3, 5]
results = []
for factor in scale_factors:
    rho = run_folded_circuit(base_error, factor, base_sequence=BASE_SEQUENCE)
    results.append(expectation_p0(rho))

slope, intercept = linear_fit(scale_factors, results)
svg_path = Path('zne_results.svg')
generate_svg(scale_factors, results, slope, intercept, svg_path)

print('Zero-Noise Extrapolation demo (standard library simulation)')
print(f'Base depolarizing probability per gate: {base_error}')
print('Fold factor | P(|0>)')
print('-' * 28)
for factor, prob in zip(scale_factors, results):
    print(f'{factor:>5}      | {prob:.6f}')

print(f'Linear fit intercept (zero-noise): {intercept:.6f}')
print(f'Plot saved to {svg_path.name}')
